In [ ]:
!pip install pandas
!pip install spacy
!pip install nltk
!pip install networkx
!pip install pyvis
!pip install openpyxl

!python -m spacy download en_core_web_trf


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.4 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 25.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


- Run this file on Google Coolab

In [ ]:
!pip install git+https://github.com/explosion/spacy-transformers

  Cloning https://github.com/explosion/spacy-transformers to /tmp/pip-req-build-e1qy7ite
  Running command git clone --filter=blob:none --quiet https://github.com/explosion/spacy-transformers /tmp/pip-req-build-e1qy7ite
  Resolved https://github.com/explosion/spacy-transformers to commit 40ee09d9b2b2b18f77fc3715329ce080691b6af9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 80.6 MB/s eta 0:00:00
  Created wheel for spacy-transformers: filename=spacy_transformers-1.3.6-cp310-cp310-linux_x86_64.whl size=680956 sha256=b6c4b26a1b1dc9811059c769a5ddfc3558f4bef606851c5b10eed0db09759c26
  Stored in directory: /tmp/pip-ephem-wheel-cache-udydc56q/wheels/83/b0/36/8f14149b7b5103f48d97f27c0ee666b64517

In [ ]:
import pandas as pd
import re
import spacy
from nltk.tokenize import sent_tokenize
import networkx as nx
from pyvis.network import Network

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### For transcripts data

In [ ]:
# Load the Excel file
# excel_path = "path_to_your_excel_file.xlsx"  # Update this to your actual path
df = pd.read_csv('/content/merged_transcripts4.csv')

In [ ]:
df

,Speaker,Dialogue
0,Monica,There's nothing to tell! He's just some guy I ...
1,Joey,"C'mon, you're going out with the guy! There's ..."
2,Chandler,"All right Joey, be nice. So does he have a hum..."
3,Phoebe,"Wait, does he eat chalk?"
4,Phoebe,"Just, 'cause, I don't want her to go through w..."
...,...,...
58655,Ross,I think I know too but I've been really wrong ...
58656,Ross,I'm sorry... we... we can't.
58657,Charlie,"All right, all right."
58658,Ross,"I mean, you just went out with my best friend,..."


In [ ]:
# Check for missing values in the 'Dialogue' column
print(df['Dialogue'].isnull().sum())


23


In [ ]:
missing_rows = df[df['Dialogue'].isnull()]
missing_rows

,Speaker,Dialogue
4893,Rachel,NaN
10658,Susan,NaN
10852,Note,NaN
18078,Sergei,NaN
22002,Monica,NaN
22709,Joey,NaN
22800,Rachel,NaN
30341,Monica,NaN
34394,Phoebe,NaN
34562,Joey,NaN


In [ ]:
# # Fill missing values with an empty string or drop them
# df['Dialogue'] = df['Dialogue'].fillna('')

In [ ]:
df = df.dropna(subset=['Dialogue'])

In [ ]:
df.shape

(58637, 2)

In [ ]:
import re

# Function to clean the dialogues
def clean_dialogue(text):
    # 1. Remove everything within braces or brackets {} or [] or ()
    text = re.sub(r'\[.*?\]|\{.*?\}|\(.*?\)', '', text)

    # 2. Remove everything before ] or } or )
    text = re.sub(r'.*[\]\}\)]', '', text)

    # 3. Remove everything after [, {, (
    text = re.sub(r'[\[\{\(].*', '', text)

    # 4. Remove all \n and other escape characters, including those in between
    text = text.replace('\n', ' ').strip()

    # 5. Remove extra spaces that may be left behind after removing \n
    text = re.sub(r'\s+', ' ', text)

    return text

df['Dialogue'] = df['Dialogue'].apply(clean_dialogue)

<ipython-input-10-904295e7941b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dialogue'] = df['Dialogue'].apply(clean_dialogue)


In [ ]:
def load_model():
    if spacy.prefer_gpu():
        print("Using GPU")
    else:
        print("Using CPU")
    nlp = spacy.load("en_core_web_trf")
    return nlp

nlp_model = load_model()

# Function to perform NER inference on dialogue
def get_ners_inference(dialogue):
    script_sentences = sent_tokenize(dialogue)
    ner_output = []

    for sentence in script_sentences:
        doc = nlp_model(sentence)
        ners = set()

        for entity in doc.ents:
            if entity.label_ == "PERSON":
                first_name = entity.text.split(" ")[0].strip()
                ners.add(first_name)

        ner_output.append(ners)

    return ner_output

# Apply NER to the Dialogue column
df['ners'] = df['Dialogue'].apply(get_ners_inference)

Using GPU


/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [ ]:
df['ners']

,ners
0,"[{}, {}]"
1,"[{}, {}]"
2,"[{Joey}, {}, {}]"
3,[{}]
4,[{}]
...,...
58655,[{}]
58656,[{}]
58657,[{}]
58658,"[{}, {}]"


### ners_column_analysis

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['ners'].sample(15)

,ners
3948,[{}]
45616,[{}]
2785,[{Ohio}]
18234,"[{}, {}]"
20066,"[{7:15}, {}]"
25291,"[{}, {the last minute}, {}]"
18649,"[{}, {}, {}]"
55488,[{}]
910,[{Ross}]
14903,[{}]


In [ ]:
# Count how many entries have no named entities
empty_count = df['ners'].apply(lambda x: len(x) == 1 and not x[0]).sum()
total_entries = len(df)

print(f"Total entries: {total_entries}")
print(f"Entries with no named entities: {empty_count}")
print(f"Percentage of empty entries: {empty_count / total_entries * 100:.2f}%")


Total entries: 58637
Entries with no named entities: 24954
Percentage of empty entries: 42.56%


In [ ]:
# Count Entity Types
from collections import Counter

# Function to extract entities
def extract_entities(ner_list):
    entities = []
    for entity_set in ner_list:
        entities.extend(entity_set)  # Directly add elements of the set to the list
    return list(entities)  # Return as a list

# Apply the function to extract entities
all_entities = df['ners'].apply(extract_entities).explode().dropna().tolist()

# Count the occurrences of each entity
entity_counts = Counter(all_entities)

# Convert counts to a DataFrame for better readability
entity_counts_df = pd.DataFrame(entity_counts.items(), columns=['Entity', 'Count']).sort_values(by='Count', ascending=False)

print(entity_counts_df)


                Entity  Count
6                 Ross   1796
0                 Joey   1511
12            Chandler   1216
9               Monica   1167
8               Rachel   1113
...                ...    ...
2297  this one weekend      1
2299    late one night      1
2300    a ton of money      1
2301         Pitstains      1
5204           African      1

[5205 rows x 2 columns]


In [ ]:
# Unique entities
unique_entities = set(all_entities)

print(f"Unique entities found: {len(unique_entities)}")
print(unique_entities)


Unique entities found: 5205
{'A MONTH', 'Jeffery', 'Nip', 'two months', 'Daytona', 'Italian-American', 'fifty cents', 'Whitney', 'Murlo', 'Second Girl', 'just about two', 'two hours ago', 'gleba', 'The Dry Cleaner', 'like a week', 'Romania', 'Professor', 'Ambercrombie & Fitch', 'Brady’s Pub', 'Logan’s Run', 'Europe', 'Africa', 'J.T.', 'her wedding day', 'Road Rules', 'James Campbell', 'Tuesday', 'A year', '25 years', 'millions and millions', 'Mary Ellen', 'May 12th', 'Soapie', 'Aramis', 'Diane Keaton', 'Wealer', 'Twenty percent', 'a weekend', 'half-Swedish', 'The Wedding March', '12.75', 'Big Leon', 'more than a year', 'Cybill Shepard', 'Slinky', 'the Wintergarden Theater', 'a whole weekend', 'Days', 'Allessandro', 'her year', 'that really', 'Patrick', 'ninety-seven', "6 o'clock in the morning", 'the Mattress King', 'Frank JR. JR.', 'Potato', 'Mississ-Pete', 'The Cute Guy', 'Althea', 'Dr. Geller', 'Red Ross!', 'Shannon Cooper', 'Rollos', 'Christmas Eve Eve', 'Waltham', 'Three hundred a

In [ ]:
pd.reset_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [ ]:
# Create a DataFrame to aggregate entity appearances by speaker
entity_appearance = {}

for index, row in df.iterrows():
    speaker = row['Speaker']
    entities = extract_entities(row['ners'])

    for entity in entities:
        if entity not in entity_appearance:
            entity_appearance[entity] = Counter()
        entity_appearance[entity][speaker] += 1

# Convert to DataFrame
entity_appearance_df = pd.DataFrame(dict(entity_appearance)).fillna(0)
print(entity_appearance_df)


### Generating the chracter network

In [ ]:
# Function to generate character network
def generate_character_network(df):
    windows = 10
    entity_relationship = []

    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        speaker = row['Speaker']  # Get the speaker's name
        current_entities = row['ners']  # Get NER entities from the dialogue

        previous_entities_in_window = []

        for entity_set in current_entities:
            previous_entities_in_window.append(list(entity_set))
            previous_entities_in_window = previous_entities_in_window[-windows:]

            previous_entities_flattened = sum(previous_entities_in_window, [])

            for entity in entity_set:
                # Create relationships with previous entities and the speaker
                for entity_in_window in previous_entities_flattened:
                    if entity != entity_in_window:
                        entity_relationship.append(sorted([speaker, entity_in_window]))

    relationship_df = pd.DataFrame({'value': entity_relationship})
    relationship_df['source'] = relationship_df['value'].apply(lambda x: x[0])
    relationship_df['target'] = relationship_df['value'].apply(lambda x: x[1])
    relationship_df = relationship_df.groupby(['source', 'target']).count().reset_index()
    relationship_df = relationship_df.sort_values('value', ascending=False)

    return relationship_df


In [ ]:
df.head(4)

,Speaker,Dialogue,ners
0,Monica,There's nothing to tell! He's just some guy I ...,"[{}, {}]"
1,Joey,"C'mon, you're going out with the guy! There's ...","[{}, {}]"
2,Chandler,"All right Joey, be nice. So does he have a hum...","[{Joey}, {}, {}]"
3,Phoebe,"Wait, does he eat chalk?",[{}]


In [ ]:
# Generate the relationship DataFrame
relationship_df1 = generate_character_network(df)

In [ ]:
relationship_df1.shape

(1348, 3)

In [ ]:
pd.set_option('display.max_rows',None)
relationship_df1.head(200)

relationship_df1.to_csv('relationship_df1.csv', index=False)

In [ ]:
relationship_df_drop = relationship_df1[relationship_df1['source'] == relationship_df1['target']]
relationship_df_drop

,source,target,value
1041,Monica,Monica,87
793,Joey,Joey,84
1140,Phoebe,Phoebe,75
1284,Ross,Ross,75
1213,Rachel,Rachel,66
251,Chandler,Chandler,38
1274,Richard,Richard,30
569,Eric,Eric,5
49,Amy,Amy,4
1000,Mike,Mike,3


In [ ]:
pd.reset_option('display.max_rows',None)

In [ ]:
relationship_df1

,source,target,value
1225,Rachel,Ross,265
1057,Monica,Rachel,244
1052,Monica,Phoebe,213
848,Joey,Rachel,211
1063,Monica,Ross,196
...,...,...,...
148,Billy,Monica,1
147,Billy,Chandler,1
901,Julie,Thomas,1
902,Julio,Monica,1


In [ ]:
relationship_df1 = relationship_df1[relationship_df1['source'] != relationship_df1['target']]

In [ ]:
pd.reset_option('display.max_rows',None)
relationship_df1

,source,target,value
1225,Rachel,Ross,265
1057,Monica,Rachel,244
1052,Monica,Phoebe,213
848,Joey,Rachel,211
1063,Monica,Ross,196
...,...,...,...
148,Billy,Monica,1
147,Billy,Chandler,1
901,Julie,Thomas,1
902,Julio,Monica,1


In [ ]:
relationship_df1.shape

(1326, 3)

In [ ]:
# pd.set_option('display.max_rows',None)
# relationship_df1.head(200)

relationship_df1.to_csv('relationship_df1.csv', index=False)

In [ ]:
relationship_df1[(relationship_df1['source'] == 'Phoebe')]

,source,target,value
10,Phoebe,Rachel,160
11,Phoebe,Ross,153
40,Phoebe,Story,21
59,Phoebe,Rachel,15
70,Phoebe,Waltham,14
...,...,...,...
982,Phoebe,Tony,1
985,Phoebe,Joey,1
1153,Phoebe,Ross,1
1154,Phoebe,Phoebe,1


### Network plotting

In [ ]:
relationship_df3 = relationship_df2.head(200)

In [ ]:
import pandas as pd

In [ ]:
relationship_df3 = pd.read_csv('/content/relationship_df_cleaner_nw_plotting.csv')

In [ ]:
relationship_df3  = relationship_df3.head(200)

In [ ]:
relationship_df3.shape

(200, 3)

In [ ]:
relationship_df3.head(20)

,value,source,target
0,323,Rachel,Ross
1,286,Monica,Rachel
2,263,Monica,Ross
3,249,Joey,Rachel
4,241,Monica,Phoebe
5,220,Joey,Ross
6,210,Chandler,Monica
7,201,Chandler,Ross
8,197,Chandler,Joey
9,189,Joey,Monica


### testing network apperance

In [ ]:
!pip install pyvis

In [ ]:
# Install necessary package
# Create a graph from the relationship DataFrame

import networkx as nx
from pyvis.network import Network

# Create the graph from the relationship dataframe
G = nx.from_pandas_edgelist(
    relationship_df3,
    source='source',
    target='target',
    edge_attr='value',
    create_using=nx.Graph()
)

# Visualize the network
net = Network(notebook=True, width="1000px", height="700px", bgcolor="#222222", font_color="white", cdn_resources="remote")

# Set node size based on their degree
node_degree = dict(G.degree)
nx.set_node_attributes(G, node_degree, 'size')

# Add the networkx graph to pyvis
net.from_nx(G)

# Inject custom JavaScript for edge highlighting and displaying relation info inside the network canvas
highlight_js = """
<script type="text/javascript">
    // Create a label for displaying relationship information within the network canvas
    var relationLabel = document.createElement('div');
    relationLabel.id = 'relation-info';
    relationLabel.style.position = 'absolute';
    relationLabel.style.top = '10px';  // Adjust this value to control position within the canvas
    relationLabel.style.right = '10px'; // Adjust to control placement
    relationLabel.style.padding = '12px'; // Increased padding for better visibility
    relationLabel.style.backgroundColor = 'rgba(50, 50, 50, 0.8)';
    relationLabel.style.color = 'white';
    relationLabel.style.fontSize = '16px'; // Increased font size for better readability
    relationLabel.style.borderRadius = '5px';
    relationLabel.style.zIndex = '999';
    relationLabel.innerHTML = 'Click on a branch to see relation info';
    document.body.appendChild(relationLabel);

    // Handle node click event
    network.on('click', function(properties) {
        // Reset all edges to the original color
        network.body.data.edges.update(network.body.data.edges.get().map(edge => ({ id: edge.id, color: '#97C2FC' })));

        // Get clicked node
        var clickedNode = properties.nodes[0];

        if (clickedNode !== undefined) {
            // Highlight the edges connected to the clicked node in a different color
            network.body.data.edges.get().forEach(edge => {
                if (edge.from === clickedNode || edge.to === clickedNode) {
                    network.body.data.edges.update({ id: edge.id, color: 'yellow' });
                }
            });
        }
    });

    // Handle edge click event (for showing relation info inside the network)
    network.on('click', function(properties) {
        var clickedEdgeId = properties.edges[0];

        if (clickedEdgeId !== undefined) {
            // Get the clicked edge information
            var clickedEdge = network.body.data.edges.get(clickedEdgeId);
            var source = network.body.data.nodes.get(clickedEdge.from).label;
            var target = network.body.data.nodes.get(clickedEdge.to).label;
            var value = clickedEdge.value;

            // Display the relationship info in the relationLabel inside the canvas
            relationLabel.innerHTML =
                '<strong>Relationship Info:</strong><br>' +
                'Person1 : ' + source + '<br>' +
                'Person2 : ' + target + '<br>' +
                'Strength : ' + value;

            // Highlight the clicked edge in light red
            network.body.data.edges.update({ id: clickedEdgeId, color: '#FF9999' });
        }
    });
</script>
"""

# Save the network to an HTML file
net.show("friends_character3_network.html")

# Append the custom JavaScript to the HTML file
with open("friends_character3_network.html", "a") as f:
    f.write(highlight_js)


friends_character3_network.html


### cleaning the errors in the names of source and target

In [ ]:
relationship_df1

,source,target,value
1225,Rachel,Ross,265
1057,Monica,Rachel,244
1052,Monica,Phoebe,213
848,Joey,Rachel,211
1063,Monica,Ross,196
...,...,...,...
148,Billy,Monica,1
147,Billy,Chandler,1
901,Julie,Thomas,1
902,Julio,Monica,1


In [ ]:
# Generate the relationship DataFrame
# same name on both side source and target are removed after processing we got: relationship_df
relationship_df_cleaner = relationship_df1

In [ ]:
relationship_df_cleaner.shape

(1326, 3)

In [ ]:
relationship_df_cleaner.head(16)

,source,target,value
1225,Rachel,Ross,265
1057,Monica,Rachel,244
1052,Monica,Phoebe,213
848,Joey,Rachel,211
1063,Monica,Ross,196
853,Joey,Ross,191
832,Joey,Monica,183
320,Chandler,Joey,181
352,Chandler,Monica,176
375,Chandler,Ross,167


In [ ]:
relationship_df_cleaner[relationship_df_cleaner['source'] == 'GELLER']

,source,target,value
630,GELLER,Richard,41
631,GELLER,Ross,16
628,GELLER,Monica,12
632,GELLER,Roy,1
629,GELLER,Phoebe,1


In [ ]:
relationship_df_cleaner[relationship_df_cleaner['target'] == 'Phoebeball']

,source,target,value
1141,Phoebe,Phoebeball,1


In [ ]:
# relationship_df_cleaner['source'] = relationship_df_cleaner['source'].str.lower()
# relationship_df_cleaner['target'] = relationship_df_cleaner['target'].str.lower()

In [ ]:
'Story'
'Big'
Sebastian
Sebastiano
Joanne
Gunther
Johnny
Fat
Betty
Harad
Harmon
LITTLE
Aired
Iris
Hugsy
Daddy
Grrrreat
BIG
'-'
Mom
Intercom
Heath
Heathcliff
Friends
By
Lynch
Long
Love
Ma
Chi-Chi
Chick
Max
'Ju'
Cheatum'
Matt = Jon Favreau
Papasifakis'
Josh-u-a
'Bitsy', 'Bert', 'Best'
'Missy'
Oprah,
'P.S.'
'Fun'
'Transcribed', 'Howe'
Barbara', 'Barbra',
'Grumpus', 'Hugh', 'Jean-Claude', '
'Je',
'Al',
'Ann',
 'Rich',
 'Ho', 'Gunnersens', 'Guru'
 'Gottliebs', 'Gee', 'George', 'Hope',
'G.Stephanopoulos', 'Gabe'
Friend', 'G.I.'
Emma-Wemma-Demma'

In [ ]:
relationship_df_cleaner['target'].unique()[400:]

array(['Spafford', 'Tanya', 'Tribianni', 'Wally', 'Ryan', 'Ryder',
       'Scotch', 'Silvia', 'HORTON', 'Myron', 'Right', 'Lipson',
       'nonononono', 'Jane'], dtype=object)

In [ ]:
rename_dict1 = {
    'Rust': 'Ross',
    'Greene': 'Rachel',
    'Monicas': 'Monica',
    'MONICA': 'Monica',
    'GREENE': 'Rachel',
    'Phoebeball': 'Phoebe',
    'Phoebo': 'Phoebe',
    'Susie': 'Susan',
    'Rosss': 'Ross',
    'Susanne': 'Susan',
    'Trib': 'Joey',
    #'Rubella': 'Ross',
    'Tribianni': 'Joey'
}


In [ ]:
rename_dict2 = {
    'Buffay': 'Phoebe',
    'Bugs': 'Phoebe',
    'RACHEL': 'Rachel',
    'Tribbiani': 'Joey',
    'Pheeboh': 'Phoebe',
    'Rossie': 'Ross',
    'Phoebs': 'Phoebe',
    'Phoe': 'Phoebe',
    'Russ': 'Ross',
    'ROSS': 'Ross',
    'sus': 'Susan',
    'Raquel': 'Ross',
    'Sue': 'Susan',
    'SUSAN': 'Susan',
    'Rust': 'Ross',
    'bing': 'Chandler',
    'Rosstepher': 'Ross',
    'Phoebe-Estelle': 'Phoebe',
    'R-R-Richard': 'Richard',
    'Rossel': 'Ross'
}

In [ ]:
rename_dict3 = {
    'Greene': 'Rachel',
    'Chan': 'Chandler',
    'Jo': 'Joey',
    'Monican': 'Monica',
    'CAROL': 'Carol',
    'Chanlder': 'Chandler',
    'Chappy': 'Chandler',
    'Chad': 'Chandler',
    'Chander': 'Chandler',
    'Phoebo': 'Phoebe',
    'Phoebs': 'Phoebe',
    'Monican': 'Monica',
    'Joel': 'Joey',
    'BUFFAY': 'Phoebe',
    'Jennifer': 'Rachel',
    'Joan': 'Joey'
}

In [ ]:
rename_dict4 = {
    'GELLER': 'Ross',
    'Geller': 'Ross',
    #'Darling': 'Chandler',
    'Pheeboh': 'Phoebe',
    'Rach': 'Rachel',
    'Bing': 'Chandler',
    'Mon': 'Monica',
    'Green': 'Rachel',
    'Pheebs': 'Phoebe',
    'Moncia': 'Monica',
    'JOEY': 'Joey',
    'CHANDLER': 'Chandler',
    'MONICA': 'Monica',
    'PHOEBE': 'Phoebe',
    'Gellers': 'Ross',
    'Joe': 'Joey',
    'GREEN': 'Rachel'
    # Add more name mappings as needed
}


In [ ]:
# Rename names in both 'source' and 'target' columns using the same dictionary
relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict1)
relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict1)

<ipython-input-196-010fc25bfb7a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict1)
<ipython-input-196-010fc25bfb7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict1)


In [ ]:
# Rename names in both 'source' and 'target' columns using the same dictionary
relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict2)
relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict2)

<ipython-input-197-dfe2aa7ae58f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict2)
<ipython-input-197-dfe2aa7ae58f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict2)


In [ ]:
# Rename names in both 'source' and 'target' columns using the same dictionary
relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict3)
relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict3)

<ipython-input-198-53f7c06496c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict3)
<ipython-input-198-53f7c06496c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict3)


In [ ]:
# Rename names in both 'source' and 'target' columns using the same dictionary
relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict4)
relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict4)

<ipython-input-199-cdd16a13a23e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['source'] = relationship_df_cleaner['source'].replace(rename_dict4)
<ipython-input-199-cdd16a13a23e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relationship_df_cleaner['target'] = relationship_df_cleaner['target'].replace(rename_dict4)


In [ ]:
relationship_df_cleaner.shape

(1326, 3)

In [ ]:
relationship_df_cleaner[relationship_df_cleaner['source'] == 'GELLER']

,source,target,value


In [ ]:
relationship_df_cleaner[relationship_df_cleaner['source'] == relationship_df_cleaner['target']]

,source,target,value
653,Ross,Ross,70
631,Ross,Ross,16
869,Joey,Joey,15
150,Chandler,Chandler,7
654,Ross,Ross,6
1285,Ross,Ross,6
965,Monica,Monica,6
1211,Rachel,Rachel,6
671,Rachel,Rachel,5
633,Rachel,Rachel,5


In [ ]:
relationship_df_cleaner = relationship_df_cleaner[relationship_df_cleaner['source'] != relationship_df_cleaner['target']]

In [ ]:
relationship_df_cleaner.shape

(1306, 3)

In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Phoebe')]

,source,target,value
1147,Phoebe,Rachel,160
1157,Phoebe,Ross,153
1134,Phoebe,Story,21
1135,Phoebe,Rachel,15
1186,Phoebe,Waltham,14
...,...,...,...
1176,Phoebe,Tiger,1
1177,Phoebe,Tony,1
1180,Phoebe,Joey,1
201,Phoebe,Ross,1


In [ ]:
relationship_df_cleaner.head(5)

,source,target,value
1225,Rachel,Ross,265
1057,Monica,Rachel,244
1052,Monica,Phoebe,213
848,Joey,Rachel,211
1063,Monica,Ross,196


In [ ]:
#group-by used to sum as same source and target value
relationship_df_cleaner = relationship_df_cleaner.groupby(['source', 'target'], as_index=False).sum()

In [ ]:
relationship_df_cleaner.shape

(1207, 3)

In [ ]:
relationship_df_cleaner = relationship_df_cleaner.sort_values(by='value', ascending=False)

In [ ]:
relationship_df_cleaner

,source,target,value
1089,Rachel,Ross,309
941,Monica,Rachel,273
939,Monica,Phoebe,241
750,Joey,Rachel,241
318,Chandler,Monica,210
...,...,...,...
349,Chandler,Stevie,1
789,Josh,Ross,1
347,Chandler,Stephen,1
792,Joshua,Waltham,1


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Rachel') & (relationship_df_cleaner['target'] == 'Ross')]

,source,target,value
1089,Rachel,Ross,309


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Ross') & (relationship_df_cleaner['target'] == 'Rachel')]

,source,target,value
1149,Ross,Rachel,14


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Joey') & (relationship_df_cleaner['target'] == 'Ross')]

,source,target,value
755,Joey,Ross,206


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Ross') & (relationship_df_cleaner['target'] == 'Joey')]

,source,target,value
1146,Ross,Joey,14


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Ross') & (relationship_df_cleaner['target'] == 'Monica')]

,source,target,value
1147,Ross,Monica,55


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Monica') & (relationship_df_cleaner['target'] == 'Ross')]

,source,target,value
947,Monica,Ross,208


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Rachel') & (relationship_df_cleaner['target'] == 'Joey')]

,source,target,value
1077,Rachel,Joey,8


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Joey') & (relationship_df_cleaner['target'] == 'Rachel')]

,source,target,value
750,Joey,Rachel,241


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Rachel') & (relationship_df_cleaner['target'] == 'Monica')]

,source,target,value
1078,Rachel,Monica,13


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Monica') & (relationship_df_cleaner['target'] == 'Rachel')]

,source,target,value
941,Monica,Rachel,273


### interesting Insight

In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Phoebe') & (relationship_df_cleaner['target'] == 'Rachel')]

,source,target,value
1022,Phoebe,Rachel,185


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Rachel') & (relationship_df_cleaner['target'] == 'Phoebe')]

,source,target,value


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Chandler') & (relationship_df_cleaner['target'] == 'Rachel')]

,source,target,value
330,Chandler,Rachel,148


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Rachel') & (relationship_df_cleaner['target'] == 'Chandler')]

,source,target,value


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Ross') & (relationship_df_cleaner['target'] == 'Chandler')]

,source,target,value
1145,Ross,Chandler,1


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Chandler') & (relationship_df_cleaner['target'] == 'Ross')]

,source,target,value
337,Chandler,Ross,200


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Monica') & (relationship_df_cleaner['target'] == 'Chandler')]

,source,target,value


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Chandler') & (relationship_df_cleaner['target'] == 'Monica')]

,source,target,value
318,Chandler,Monica,210


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Chandler') & (relationship_df_cleaner['target'] == 'Joey')]

,source,target,value
288,Chandler,Joey,197


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Joey') & (relationship_df_cleaner['target'] == 'Chandler')]

,source,target,value


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Joey') & (relationship_df_cleaner['target'] == 'Chandler')]

,source,target,value


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Ross') & (relationship_df_cleaner['target'] == 'Phoebe')]

,source,target,value
1148,Ross,Phoebe,5


In [ ]:
relationship_df_cleaner[(relationship_df_cleaner['source'] == 'Phoebe') & (relationship_df_cleaner['target'] == 'Ross')]

,source,target,value
1032,Phoebe,Ross,170


### unifying the source and target: interchangable

In [ ]:
# Step 1: Create a sorted tuple of source and target to treat them as undirected edges
relationship_df_cleaner['edge'] = relationship_df_cleaner.apply(lambda row: tuple(sorted([row['source'], row['target']])), axis=1)
relationship_df_cleaner['edge']

,edge
1089,"(Rachel, Ross)"
941,"(Monica, Rachel)"
939,"(Monica, Phoebe)"
750,"(Joey, Rachel)"
318,"(Chandler, Monica)"
...,...
349,"(Chandler, Stevie)"
789,"(Josh, Ross)"
347,"(Chandler, Stephen)"
792,"(Joshua, Waltham)"


In [ ]:
# Step 2: Group by these sorted edges and sum their values
relationship_df_cleaner = relationship_df_cleaner.groupby('edge', as_index=False).agg({'value': 'sum'})
relationship_df_cleaner

,edge,value
0,"(-, Amy)",4
1,"(Aaron, By)",2
2,"(Aaron, Joey)",2
3,"(Adam, Phoebe)",1
4,"(Adam, Rachel)",1
...,...,...
1191,"(Susan, nonononono)",1
1192,"(Tag, Zelner)",1
1193,"(Ted, Teleplay)",1
1194,"(Theodore, Tom)",2


In [ ]:
# Step 3: Split the sorted tuples back into 'source' and 'target'
relationship_df_cleaner[['source', 'target']] = pd.DataFrame(relationship_df_cleaner['edge'].tolist(), index=relationship_df_cleaner.index)
relationship_df_cleaner

,edge,value,source,target
0,"(-, Amy)",4,-,Amy
1,"(Aaron, By)",2,Aaron,By
2,"(Aaron, Joey)",2,Aaron,Joey
3,"(Adam, Phoebe)",1,Adam,Phoebe
4,"(Adam, Rachel)",1,Adam,Rachel
...,...,...,...,...
1191,"(Susan, nonononono)",1,Susan,nonononono
1192,"(Tag, Zelner)",1,Tag,Zelner
1193,"(Ted, Teleplay)",1,Ted,Teleplay
1194,"(Theodore, Tom)",2,Theodore,Tom


In [ ]:
# Step 4: Drop the 'edge' column as it's no longer needed
relationship_df_cleaner = relationship_df_cleaner.drop(columns='edge')
relationship_df_cleaner

,value,source,target
0,4,-,Amy
1,2,Aaron,By
2,2,Aaron,Joey
3,1,Adam,Phoebe
4,1,Adam,Rachel
...,...,...,...
1191,1,Susan,nonononono
1192,1,Tag,Zelner
1193,1,Ted,Teleplay
1194,2,Theodore,Tom


In [ ]:
# Now `relationship_df_cleaned` has merged rows and summed values
relationship_df_cleaner.head()

,value,source,target
0,4,-,Amy
1,2,Aaron,By
2,2,Aaron,Joey
3,1,Adam,Phoebe
4,1,Adam,Rachel


In [ ]:
# Sort the relationship_df_cleaned DataFrame by 'value' in descending order
relationship_df_cleaner = relationship_df_cleaner.sort_values(by='value', ascending=False)

# Display the first few rows of the sorted DataFrame
relationship_df_cleaner.head(30)


,value,source,target
1085,323,Rachel,Ross
940,286,Monica,Rachel
946,263,Monica,Ross
750,249,Joey,Rachel
938,241,Monica,Phoebe
755,220,Joey,Ross
318,210,Chandler,Monica
337,201,Chandler,Ross
288,197,Chandler,Joey
739,189,Joey,Monica


In [ ]:
relationship_df_cleaner.shape

(1196, 3)

In [ ]:
#export the file and save it and input this file to the netowrk plotting , before limit number of rows using .head(200)

relationship_df_cleaner.to_csv('relationship_df_cleaner_nw_plotting.csv', index=False)

In [ ]:
relationship_df2 = relationship_df_cleaner

#go to network plotting

In [ ]:
relationship_df2

,value,source,target
1085,323,Rachel,Ross
940,286,Monica,Rachel
946,263,Monica,Ross
750,249,Joey,Rachel
938,241,Monica,Phoebe
...,...,...,...
779,1,Joey,Woman
780,1,Joey,Wong
346,1,Chandler,Snoopy
344,1,Chandler,Sid
